In [5]:
pip install spectral

Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install -e .

Obtaining file:///home/jovyan/data-store/cross-sensor-cal
  Preparing metadata (setup.py) ... done
  Attempting uninstall: cross_sensor_cal
    Found existing installation: cross_sensor_cal 0.1
    Uninstalling cross_sensor_cal-0.1:
      Successfully uninstalled cross_sensor_cal-0.1
  DEPRECATION: Legacy editable install of cross_sensor_cal==0.1 from file:///home/jovyan/data-store/cross-sensor-cal (setup.py develop) is deprecated. pip 25.3 will enforce this behaviour change. A possible replacement is to add a pyproject.toml or enable --use-pep517, and use setuptools >= 64. If the resulting installation is not behaving as expected, try using --config-settings editable_mode=compat. Please consult the setuptools documentation for more information. Discussion can be found at https://github.com/pypa/pip/issues/11457
  Running setup.py develop for cross_sensor_cal
Note: you may need to restart the kernel to use updated packages.


In [7]:
cp base.py /opt/conda/envs/macrosystems/lib/python3.10/site-packages/hytools/base.py

In [5]:
import src as el_spectral
import bin as bin
from bin.jefe import jefe

# Updated inputs
base_folder = "NIWO_2023_07"
site_code = "NIWO"
year_month = "2023-08"
flight_lines = [
    "NEON_D13_NIWO_DP1_L019-1_20230815_directional_reflectance"
]
polygon_layer_path = "Datasets/niwot_aop_polygons_2023_12_8_23_analysis_ready_half_diam.gpkg"  # <-- Update this to your actual polygon file

# Run jefe
jefe(
    base_folder=base_folder,
    site_code=site_code,
    year_month=year_month,
    flight_lines=flight_lines,
    polygon_layer_path=polygon_layer_path,
    remote_prefix="",  # Optional: leave empty or set your iRODS subpath
    sync_files=True    # Set to False if you want to skip syncing for now
)

ImportError: cannot import name 'flight_lines_to_envi' from 'src.neon_to_envi' (/home/jovyan/data-store/cross-sensor-cal/src/neon_to_envi.py)

In [8]:
from pathlib import Path
from src.envi_download import download_neon_flight_lines
from src.neon_to_envi import neon_to_envi
from src.topo_and_brdf_correction import generate_config_json, topo_and_brdf_correction
from src.file_types import NEONReflectanceConfigFile

# Inputs
base_folder = Path("clean_test")
site_code = "NIWO"
product_code = "DP1.30006.001"
year_month = "2023-08"
flight_lines = [
    "NEON_D13_NIWO_DP1_L019-1_20230815_directional_reflectance",
    "NEON_D13_NIWO_DP1_L020-1_20230815_directional_reflectance"
]

#Step 1: Download NEON flight lines
print("📥 Downloading NEON flight lines...")
download_neon_flight_lines(
    out_dir=base_folder,
    site_code=site_code,
    product_code=product_code,
    year_month=year_month,
    flight_lines=flight_lines
)
print("✅ Download complete.\n")

# Step 2: Convert H5 to ENVI format (direct call to neon_to_envi)
print("📦 Converting H5 files to ENVI format...")
h5_files = list(base_folder.rglob("*.h5"))
if not h5_files:
    print("❌ No .h5 files found for conversion.")
else:
    for i, h5_file in enumerate(h5_files, start=1):
        print(f"🔄 [{i}/{len(h5_files)}] Converting: {h5_file.name}")
        neon_to_envi(
            images=[str(h5_file)],
            output_dir=str(base_folder),
            anc=True
        )
        print(f"✅ Finished: {h5_file.name}\n")
print("✅ ENVI conversion complete.\n")

# Step 3: Generate configuration JSON
print("📝 Generating configuration JSON...")
generate_config_json(base_folder)
print("✅ Config JSON generation complete.\n")

# Step 4: Apply topographic and BRDF corrections
print("⛰️ Applying topographic and BRDF corrections...")

# Find all JSON config files
config_files = NEONReflectanceConfigFile.find_in_directory(Path(base_folder))

if config_files:
    for config_file in config_files:
        print(f"⚙️ Applying corrections to: {config_file.file_path}")
        topo_and_brdf_correction(str(config_file.file_path))
    print("✅ All corrections applied.\n")
else:
    print("❌ No configuration JSON files found. Skipping corrections.")

from src.convolution_resample import resample
from src.file_types import NEONReflectanceBRDFCorrectedENVIFile

# Step 5: Resample and translate data to other sensor formats
print("🔁 Resampling and translating data...")

# Find all BRDF-corrected ENVI files in the base folder
corrected_files = NEONReflectanceBRDFCorrectedENVIFile.find_in_directory(base_folder)
if not corrected_files:
    print("❌ No BRDF-corrected ENVI files found for resampling. Check naming or previous steps.")
else:
    print(f"📂 Found {len(corrected_files)} BRDF-corrected files to process.")
    for i, corrected_file in enumerate(corrected_files, start=1):
        print(f"🔄 [{i}/{len(corrected_files)}] Resampling: {corrected_file.name}")
        resample(corrected_file.directory)
        print(f"✅ Resampled: {corrected_file.name}\n")

print("✅ Resampling and translation complete.\n")



print("🎉 Pipeline complete!")


📥 Downloading NEON flight lines...
Skipping clean_test/NEON_D13_NIWO_DP1_L019-1_20230815_directional_reflectance.h5, already exists
Skipping clean_test/NEON_D13_NIWO_DP1_L020-1_20230815_directional_reflectance.h5, already exists
✅ Download complete.

📦 Converting H5 files to ENVI format...
🔄 [1/2] Converting: NEON_D13_NIWO_DP1_L019-1_20230815_directional_reflectance.h5


2025-10-03 18:34:26,219	WARNING services.py:1996 -- WARNING: The object store is using /tmp instead of /dev/shm because /dev/shm has only 67108864 bytes available. This will harm performance! You may be able to free up space by deleting files in /dev/shm. If you are inside a Docker container, you can increase /dev/shm size by passing '--shm-size=10.24gb' to 'docker run' (or add it to the run_options list in a Ray cluster config). Make sure to set this to more than 30% of available RAM.
2025-10-03 18:34:27,273	INFO worker.py:1673 -- Started a local Ray instance.



📦 Exporting ancillary ENVI data...
(HyTools pid=40951) ⚠️ Skipping existing file: clean_test/NEON_D13_NIWO_DP1_L019-1_20230815_directional_reflectance/NEON_D13_NIWO_DP1.30006.001_L019-1_20230815_directional_reflectance_envi.img
✅ All processing complete.
✅ Finished: NEON_D13_NIWO_DP1_L019-1_20230815_directional_reflectance.h5

🔄 [2/2] Converting: NEON_D13_NIWO_DP1_L020-1_20230815_directional_reflectance.h5


2025-10-03 18:36:00,501	WARNING services.py:1996 -- WARNING: The object store is using /tmp instead of /dev/shm because /dev/shm has only 67108864 bytes available. This will harm performance! You may be able to free up space by deleting files in /dev/shm. If you are inside a Docker container, you can increase /dev/shm size by passing '--shm-size=10.24gb' to 'docker run' (or add it to the run_options list in a Ray cluster config). Make sure to set this to more than 30% of available RAM.
2025-10-03 18:36:01,566	INFO worker.py:1673 -- Started a local Ray instance.



📦 Exporting ancillary ENVI data...
(HyTools pid=41795) ⚠️ Skipping existing file: clean_test/NEON_D13_NIWO_DP1_L020-1_20230815_directional_reflectance/NEON_D13_NIWO_DP1.30006.001_L020-1_20230815_directional_reflectance_envi.img
✅ All processing complete.
✅ Finished: NEON_D13_NIWO_DP1_L020-1_20230815_directional_reflectance.h5

✅ ENVI conversion complete.

📝 Generating configuration JSON...
📝 Generating configuration JSON (HyTools-ready)...
📂 Found 2 reflectance files.
✅ Config saved: clean_test/NEON_D13_NIWO_DP1_L019-1_20230815_directional_reflectance/NEON_D13_NIWO_DP1.30006.001_L019-1_20230815_directional_reflectance_envi_config_envi.json
✅ Config saved: clean_test/NEON_D13_NIWO_DP1_L020-1_20230815_directional_reflectance/NEON_D13_NIWO_DP1.30006.001_L020-1_20230815_directional_reflectance_envi_config_envi.json
✅ Config JSON generation complete.

⛰️ Applying topographic and BRDF corrections...
⚙️ Applying corrections to: clean_test/NEON_D13_NIWO_DP1_L019-1_20230815_directional_reflect

2025-10-03 18:37:58,821	WARNING services.py:1996 -- WARNING: The object store is using /tmp instead of /dev/shm because /dev/shm has only 67108864 bytes available. This will harm performance! You may be able to free up space by deleting files in /dev/shm. If you are inside a Docker container, you can increase /dev/shm size by passing '--shm-size=10.24gb' to 'docker run' (or add it to the run_options list in a Ray cluster config). Make sure to set this to more than 30% of available RAM.
2025-10-03 18:37:59,882	INFO worker.py:1673 -- Started a local Ray instance.


📦 Ancillary mapping for HyTools:
{
  "clean_test/NEON_D13_NIWO_DP1_L019-1_20230815_directional_reflectance/NEON_D13_NIWO_DP1.30006.001_L019-1_20230815_directional_reflectance_envi.img": {
    "path_length": [
      "clean_test/NEON_D13_NIWO_DP1_L019-1_20230815_directional_reflectance/NEON_D13_NIWO_DP1.30006.001_L019-1_20230815_directional_reflectance_ancillary_envi.img",
      0
    ],
    "sensor_az": [
      "clean_test/NEON_D13_NIWO_DP1_L019-1_20230815_directional_reflectance/NEON_D13_NIWO_DP1.30006.001_L019-1_20230815_directional_reflectance_ancillary_envi.img",
      1
    ],
    "sensor_zn": [
      "clean_test/NEON_D13_NIWO_DP1_L019-1_20230815_directional_reflectance/NEON_D13_NIWO_DP1.30006.001_L019-1_20230815_directional_reflectance_ancillary_envi.img",
      2
    ],
    "solar_az": [
      "clean_test/NEON_D13_NIWO_DP1_L019-1_20230815_directional_reflectance/NEON_D13_NIWO_DP1.30006.001_L019-1_20230815_directional_reflectance_ancillary_envi.img",
      3
    ],
    "solar_zn":

2025-10-03 18:38:06,012	WARNING services.py:1996 -- WARNING: The object store is using /tmp instead of /dev/shm because /dev/shm has only 67108864 bytes available. This will harm performance! You may be able to free up space by deleting files in /dev/shm. If you are inside a Docker container, you can increase /dev/shm size by passing '--shm-size=10.24gb' to 'docker run' (or add it to the run_options list in a Ray cluster config). Make sure to set this to more than 30% of available RAM.
2025-10-03 18:38:07,068	INFO worker.py:1673 -- Started a local Ray instance.


📦 Ancillary mapping for HyTools:
{
  "clean_test/NEON_D13_NIWO_DP1_L020-1_20230815_directional_reflectance/NEON_D13_NIWO_DP1.30006.001_L020-1_20230815_directional_reflectance_envi.img": {
    "path_length": [
      "clean_test/NEON_D13_NIWO_DP1_L020-1_20230815_directional_reflectance/NEON_D13_NIWO_DP1.30006.001_L020-1_20230815_directional_reflectance_ancillary_envi.img",
      0
    ],
    "sensor_az": [
      "clean_test/NEON_D13_NIWO_DP1_L020-1_20230815_directional_reflectance/NEON_D13_NIWO_DP1.30006.001_L020-1_20230815_directional_reflectance_ancillary_envi.img",
      1
    ],
    "sensor_zn": [
      "clean_test/NEON_D13_NIWO_DP1_L020-1_20230815_directional_reflectance/NEON_D13_NIWO_DP1.30006.001_L020-1_20230815_directional_reflectance_ancillary_envi.img",
      2
    ],
    "solar_az": [
      "clean_test/NEON_D13_NIWO_DP1_L020-1_20230815_directional_reflectance/NEON_D13_NIWO_DP1.30006.001_L020-1_20230815_directional_reflectance_ancillary_envi.img",
      3
    ],
    "solar_zn":


KeyboardInterrupt



In [3]:
pip install -e .

Obtaining file:///home/jovyan/data-store/cross-sensor-cal
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for cross_sensor_cal (pyproject.toml) ... done
  Created wheel for cross_sensor_cal: filename=cross_sensor_cal-0.1.0-0.editable-py3-none-any.whl size=27465 sha256=3f85128303030bec6e45e0888944f7ebf396c09b45d225fecac1377ee852eb6e
  Stored in directory: /tmp/pip-ephem-wheel-cache-bqrde3x4/wheels/18/70/06/063f1fc66efdafe44297ee2e506086a9ff3bc04776dd2148a3
Successfully built cross_sensor_cal
  Attempting uninstall: cross_sensor_cal
    Found existing installation: cross_sensor_cal 0.1.0
    Uninstalling cross_sensor_cal-0.1.0:
      Successfully uninstalled cross_sensor_cal-0.1.0
Note: you may need to restart the kernel to use updated packages.


In [2]:
from pathlib import Path
from bin.jefe import go_forth_and_multiply


base_folder = Path("output_test")
site_code = "NIWO"
product_code = "DP1.30006.001"
year_month = "2023-08"
flight_lines = [
    "NEON_D13_NIWO_DP1_L019-1_20230815_directional_reflectance",
    "NEON_D13_NIWO_DP1_L020-1_20230815_directional_reflectance"
]

go_forth_and_multiply(
    base_folder=base_folder,
    site_code=site_code,
    year_month=year_month,
    flight_lines=flight_lines,
)


⏭️  Skipped: download already present (2)


L019-1           0%|          | 0/1 steps
                                         
L019-1         100%|██████████| 1/1 steps
                                         
L019-1         100%|██████████| 2/2 steps
L020-1         100%|██████████| 1/1 steps
                                         
L019-1         100%|██████████| 2/2 steps
L019-1          67%|██████▋   | 2/3 steps
                                         
L019-1          75%|███████▌  | 3/4 steps
L020-1         100%|██████████| 3/3 steps
                                         
L019-1         100%|██████████| 4/4 steps
L019-1          80%|████████  | 4/5 steps

⏭️  Skipped [L019-1]: ENVI + ancillary already exported (1)
⏭️  Skipped [L020-1]: ENVI + ancillary already exported (1)
⏭️  Skipped: config already present (2)
⏭️  Skipped [L019-1]: BRDF+topo correction already present (2)
⏭️  Skipped [L020-1]: BRDF+topo correction already present (2)
🚀 Starting convolutional resample for output_test/NEON_D13_NIWO_DP1_L019-1_20230815_directional_reflectance
📂 Opening: output_test/NEON_D13_NIWO_DP1_L019-1_20230815_directional_reflectance/NEON_D13_NIWO_DP1.30006.001_L019-1_20230815_directional_reflectance_brdfandtopo_corrected_envi.hdr
✅ Resampled file saved: output_test/NEON_D13_NIWO_DP1_L019-1_20230815_directional_reflectance/Convolution_Reflectance_Resample_Landsat_5_TM/NEON_D13_NIWO_DP1.30006.001_L019-1_20230815_directional_resampled_Landsat_5_TM_envi.hdr
✅ Resampled file saved: output_test/NEON_D13_NIWO_DP1_L019-1_20230815_directional_reflectance/Convolution_Reflectance_Resample_Landsat_7_ETM+/NEON_D13_NIWO_DP1.30006.001_L019-1_20230815_directional_

                                         
L019-1          83%|████████▎ | 5/6 steps
L020-1         100%|██████████| 4/4 steps
L020-1          80%|████████  | 4/5 steps

✅ Resampled file saved: output_test/NEON_D13_NIWO_DP1_L019-1_20230815_directional_reflectance/Convolution_Reflectance_Resample_MicaSense-to-match_OLI_and_OLI-2/NEON_D13_NIWO_DP1.30006.001_L019-1_20230815_directional_resampled_MicaSense-to-match_OLI_and_OLI-2_envi.hdr
ℹ️ Skipping mask file without spectral bands: output_test/NEON_D13_NIWO_DP1_L019-1_20230815_directional_reflectance/NEON_D13_NIWO_DP1.30006.001_L019-1_20230815_directional_reflectance_brdfandtopo_corrected_mask_envi.hdr
⏭️  Skipped [NEON_D13_NIWO_DP1.30006.001_L019-1_20230815_directional_reflectance_brdfandtopo_corrected_mask_envi.img]: resampled outputs already present (7)
🚀 Starting convolutional resample for output_test/NEON_D13_NIWO_DP1_L020-1_20230815_directional_reflectance
📂 Opening: output_test/NEON_D13_NIWO_DP1_L020-1_20230815_directional_reflectance/NEON_D13_NIWO_DP1.30006.001_L020-1_20230815_directional_reflectance_brdfandtopo_corrected_envi.hdr
✅ Resampled file saved: output_test/NEON_D13_NIWO_DP1_L020-1_202308


L019-1         100%|██████████| 6/6 steps
L020-1         100%|██████████| 5/5 steps

🎉 Pipeline complete!


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from spectral import open_image

def plot_envi_band(img_path: str, band_index: int = 0, cmap: str = 'gray'):
    """
    Plot a single band from an ENVI .img file.
    
    Parameters:
        img_path (str): Path to the .img file (expects corresponding .hdr next to it).
        band_index (int): Index of the band to plot (0-based).
        cmap (str): Matplotlib colormap name for grayscale display.
    """
    # Load the image
    hdr_path = img_path.replace('.img', '.hdr')
    img = open_image(hdr_path)
    
    # Extract the band
    data = np.array(img[:, :, band_index], dtype=float)
    
    # Plot
    plt.figure(figsize=(8, 8))
    plt.imshow(data, cmap=cmap)
    plt.title(f'ENVI Band {band_index}')
    plt.colorbar(label='Reflectance')
    plt.axis('off')
    plt.show()


def plot_envi_rgb(img_path: str, rgb_bands=(29, 19, 9), stretch=(2, 98)):
    """
    Plot an RGB composite from an ENVI .img file by specifying three band indices.
    
    Parameters:
        img_path (str): Path to the .img file (expects corresponding .hdr next to it).
        rgb_bands (tuple): Three band indices (R, G, B) to composite.
        stretch (tuple): Percentile stretch (e.g., (2,98)) for contrast enhancement.
    """
    hdr_path = img_path.replace('.img', '.hdr')
    img = open_image(hdr_path)
    
    # Read the three bands
    rgb = np.array(img[:, :, list(rgb_bands)], dtype=float)
    
    # Percentile stretch per channel
    p_low, p_high = np.percentile(rgb, stretch, axis=(0,1))
    rgb = (rgb - p_low) / (p_high - p_low)
    rgb = np.clip(rgb, 0, 1)
    
    plt.figure(figsize=(8, 8))
    plt.imshow(rgb)
    plt.title(f'RGB Composite (bands {rgb_bands})')
    plt.axis('off')
    plt.show()


img_path = 'later_test/NEON_D13_NIWO_DP1_L019-1_20230815_directional_reflectance/NEON_D13_NIWO_DP1_L019-1_20230815_reflectance_brdfandtopo_corrected_envi.img'
plot_envi_band(img_path, band_index=1)